## Problem 1: Fibonacci Sequence Generator

<span style="font-size:18px;">Create a generator function that yields numbers in the Fibonacci sequence up to a specified limit.</span>

In [1]:
def fibonacci_generator(limit):
    f0 = 0
    f1 = 1
    sum_f = 0

    while sum_f <= limit:
        yield sum_f
        f0 = f1
        f1 = sum_f
        sum_f = f0 + f1

In [2]:
fibonacci_generator(100)

<generator object fibonacci_generator at 0x00000221982DC740>

In [3]:
for term in fibonacci_generator(100):
    print(term)

0
1
1
2
3
5
8
13
21
34
55
89


## Problem 2: Chunking an Iterable.

<span style="font-size:18px;">Create a generator function that yields chunks of a specified size from an iterable.</span>

Example:

Input: `[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]` <br>
Output: `[0, 1, 2], [3, 4, 5], [6, 7, 8], [9]` <br>


In [4]:
# Solution 1: Most appropriate solution.

def chunker(iterable, size):
    if isinstance(size, int) == False:
        print(f'Please enter an integer value for the size in which you want to divide the iterable.')
        return

    for i in range(0, len(iterable), size):
        yield iterable[i : i + size]
        
chunked_iterables = chunker(tuple(range(0, 11)), 3)

for item in chunked_iterables:
    print(item)

(0, 1, 2)
(3, 4, 5)
(6, 7, 8)
(9, 10)


In [5]:
# Solution 2:

import itertools

def chunker(iterable, size):
    it = iter(iterable)

    while True:
        chunk = list(itertools.islice(it, size))
        if not chunk:
            return
        yield chunk

for chunk in chunker(range(0, 10), 3):
    print(chunk)

[0, 1, 2]
[3, 4, 5]
[6, 7, 8]
[9]


In [6]:
# Solution 3: 

def chunker(iterable, size):
    # Step 1: Create an iterator from the iterable
    iterator = iter(iterable)

    # Step 2: Duplicate the iterator. That is, create a list of references to the same iterator, repeated size times.
    # This results in the list [iterator, iterator, iterator], where each element refers to the same iterator object.
    iterators = [iterator] * size

    # Step 3: Use zip() function to group the elements from the iterator into tuples of length `size`.
    chunked_iterable = zip(*iterators)

    # Step 4: Convert the result to a list
    chunked_list = list(chunked_iterable)

    # Step 5: Handle the remaining elements (if any)
    remainder = list(iterator)
    if remainder:
        chunked_list.append(tuple(remainder))

    for item in chunked_list:
        yield item

chunked_items =  chunker(range(0, 10), 3)
for item in chunked_items:
    print(item)

(0, 1, 2)
(3, 4, 5)
(6, 7, 8)


```Python
iterator = iter(iterables)
iterators = [iterator] * size
chunked_iterable = zip(*iterators)
chunked_list = list(chunked_iterable)
```

The above four lines of code can be condensed into:

```Python
chunked_list = list(zip(*[iter(iterables)] * size))
```

Let us dive deeper into what is going on in this atypical code snippet.

<b>Example Setup</b> <br>
We'll use the following inputs:

* `iterable = [1, 2, 3, 4, 5, 6, 7, 8, 9]`
* `size = 3` <br><br>

1. <b>Create an iterator:</b>
    * First, create an iterator from the `iterable`. This iterator will sequentially return elements from the list `[1, 2, 3, 4, 5, 6, 7, 8, 9]`. <br><br>

2. <b>Duplicate the iterator:</b>
    * Next, create a list of `size` references to the same iterator. That is, create a list containing the same iterator object repeated `size` times. This results in the list `[iterator, iterator, iterator]`, where each element refers to the same iterator object. <br><br>

3. <b>Unpack and Zip</b>
    * Then, use `zip(*iterators)` to group elements from the iterator into tuples of length `size`.
    * The `zip()` function takes multiple iterables and aggregates elements from each iterable.
    * The `*` operator is used to unpack the list of iterators so that `zip()` receives them as separate arguments.
    * `zip()` will then take one element from each iterator in turn and form tuples. Since all iterators refer to the same underlying iterator object, `zip()` effectively takes `size` elements at a time from the single iterator and groups them into tuples.
    * During the first call (iteration), the `zip()` extracts `1`(first element) from the first reference. Then the iterator points to `2` and since all three references point to the same iterator in the memory, the second element extracted will be `2`. The iterator then points to `3`, so now, `zip()` extracts the third element, that is `3`. Thus, we get out first tuple and it will be `(1, 2, 3)`.
    * Now the iterator remembers its last state and now it points to the element `4`. So during the second call (iteration), `zip()` will extract `4` first, `5` next and `6` last, and hence, we get our second tuple `(4, 5, 6)` and so on. 

## Problem 3: Prime Number Generator
<span style="font-size:18px;">Create a generator function that yields prime numbers indefinitely (or until manually stopped).</span>

In [7]:
def prime_generator():
    D = {} # Dictionary to store the multiples of detected primes
    q = 2 # Start the primality test with the smallest known prime number, that is, 2

    while True:
        if q not in D:
            # If the above if condition becomes True then q is a new prime number
            yield q # If q is a new prime number, then yield q
            D[q * q] = [q] # Next, store its first multiple in D which hasn't been stored yet
        else:
            # If the above if condition becomes False, then q is not a prime number
            for p in D[q]: # If it is not a prime, then it must be one of the keys in D. Now loop through the values of that particular key.
                D.setdefault(p + q, []).append(p)
                # For each prime p that divides q, update the dictionary to store the next multiple of p that needs to be checked (p + q).
            del D[q] # Remove the non prime key from the dictionary as it has already been checked and ignored
        q += 1 # Increment q by 1

In [8]:
primes = prime_generator()

for _ in range(25):
    print(next(primes))

2
3
5
7
11
13
17
19
23
29
31
37
41
43
47
53
59
61
67
71
73
79
83
89
97


## Problem 4: Memoization Decorator
<span style="font-size:18px;">Create a decorator that caches the results of a function (memoization) to improve performance.</span>

In [9]:
def cache_memory(func):
    cache = {} # Declare an empty dictionary to store values which will be used as cache memory to retrieve already processed data

    def wrapper_function(*args):
        if args in cache: # If a value already stored in cache memory is asked, directly access the cache memory and skip further processing
            return cache[args]

        result = func(*args) # If new data is supplied, process it and capture its value in a variable
        cache[args] = result # Store the result of the proocessing carried out in the above step as a new entry in the cache memory

        return result # Return the result

    return wrapper_function # Return the wrapper function

In [10]:
@cache_memory
def fibonacci(term):
    if term <= 1:
        return term
    else:
        return (fibonacci(term - 1) + fibonacci(term - 2)) 

In [11]:
fibonacci(100)

354224848179261915075

If we want to see how exactly the cache memory is being loaded, we can add a `print()` statement inside `wrapper_function()`. The entire code with the output is shown below:

```Python
def cache_memory(func):
    cache = {} # Declare an empty dictionary to store values which will be used as cache memory to retrieve already processed data

    def wrapper_function(*args):
        if args in cache: # If a value already stored in cache memory is asked, directly access the cache memory and skip further processing
            return cache[args]

        result = func(*args) # If new data is supplied, process it and capture its value in a variable
        cache[args] = result # Store the result of the proocessing carried out in the above step as a new entry in the cache memory
        print(cache)
        return result # Return the result

    return wrapper_function # Return the wrapper function

@cache_memory
def fibonacci(term):
    if term <= 1:
        return term
    else:
        return (fibonacci(term - 1) + fibonacci(term - 2)) 

# Populating the cache memory: Visual display
for i in range(0, 11):
    fibonacci(i)

```

<h4>Output:</h4>

```
{(0,): 0}
{(0,): 0, (1,): 1}
{(0,): 0, (1,): 1, (2,): 1}
{(0,): 0, (1,): 1, (2,): 1, (3,): 2}
{(0,): 0, (1,): 1, (2,): 1, (3,): 2, (4,): 3}
{(0,): 0, (1,): 1, (2,): 1, (3,): 2, (4,): 3, (5,): 5}
{(0,): 0, (1,): 1, (2,): 1, (3,): 2, (4,): 3, (5,): 5, (6,): 8}
{(0,): 0, (1,): 1, (2,): 1, (3,): 2, (4,): 3, (5,): 5, (6,): 8, (7,): 13}
{(0,): 0, (1,): 1, (2,): 1, (3,): 2, (4,): 3, (5,): 5, (6,): 8, (7,): 13, (8,): 21}
{(0,): 0, (1,): 1, (2,): 1, (3,): 2, (4,): 3, (5,): 5, (6,): 8, (7,): 13, (8,): 21, (9,): 34}
{(0,): 0, (1,): 1, (2,): 1, (3,): 2, (4,): 3, (5,): 5, (6,): 8, (7,): 13, (8,): 21, (9,): 34, (10,): 55}
```

## Problem 5: Logging Decorator

<span style="font-size:20px;">Create a decorator that logs the execution time and arguments of a function each time it is called.</span>

In [12]:
import functools
import time 

def log_execution(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        t1 = time.perf_counter() # Start time (high resolution timer)
        result = func(*args, **kwargs)
        t2 = time.perf_counter() # End time (high resolution timer)
        print(f'Function {func.__name__} called with {args}, {kwargs} took {t2 - t1} seconds.')
        return result
    return wrapper

# Decorating a function which sorts a list by the Bubble Sort algorithm.
@log_execution
def bubble_sort(arr):

    none_count = arr.count(None)
    arr = [x for x in arr if x is not None]
    arr = [None] * none_count + arr

    k = len(arr) - 1

    for i in range(none_count, k, 1):
        swapped = False
        
        for j in range(none_count, k - i + none_count, 1):
            if arr[j] > arr[j + 1]:
                arr[j], arr[j + 1] = arr[j + 1], arr[j]
                swapped = True
                
        if not swapped:
            break

    return arr

# Decorating a function which sorts a list by Insertion Sort algorithm
@log_execution
def insertion_sort(arr):
    none_count = arr.count(None)
    arr = [x for x in arr if x is not None]
    arr = [None] * none_count + arr

    length = len(arr)
    
    for i in range(none_count + 1, length):
        temp = arr[i] 
        j = i - 1

        while (j >= none_count and arr[j] > temp):
            arr[j + 1] = arr[j]
            j -= 1

        arr[j + 1] = temp

    return arr

# Decorating a function which sorts a list by Selection Sort algorithm.
@log_execution
def selection_sort(data):
    none_count = data.count(None)
    data = [x for x in data if x is not None]
    data = [None] * none_count + data
    
    for i in range(none_count, len(data) - 1):
        min_idx = i

        for j in range(i + 1, len(data)):
            if data[j] < data[min_idx]:
                min_idx = j

        if min_idx != i:
            data[i], data[min_idx] = data[min_idx], data[i]

    return data

In [13]:
test_cases = [
    [] * 100,  # Empty list
    [1, 2, 3, 4, 5] * 100,  # Already sorted list
    [5, 4, 3, 2, 1] * 100,  # Reverse sorted list
    [3, 1, 4, 1, 5, 9, 2, 6, 5] * 100,  # List with duplicates
    [-3, 1, -4, 0, 2, -1] * 100,  # List with negative numbers
    [3, 0, -2, 1, 5, 0, 4] * 100,  # List with zero
    [42],  # Single element list
    [100, 20, 50, 30, 80, 10, 90, 60, 40, 70] * 100,  # Large list
    [None, 3, None, 1, 2, None] * 100,  # List with None values
    [-4, 7, 9, 0, 14, None, None, 5, 2, 10, None, -101, -5, None, 3, 1, 35] * 100, # Combination of positive, negative, zero and None values
    ['banana', 'apple', 'orange', 'grape', 'mango']  # List with strings
]

for idx, case in enumerate(test_cases):
    print(f"\nTest Case {idx + 1}: {case}")
    print()
    print("Bubble Sort:")
    bubble_sort(case.copy())
    print()
    print("Insertion Sort:")
    insertion_sort(case.copy())
    print()
    print("Selection Sort:")
    selection_sort(case.copy())
    print()
    print('-' * 100)


Test Case 1: []

Bubble Sort:
Function bubble_sort called with ([],), {} took 9.800001862458885e-06 seconds.

Insertion Sort:
Function insertion_sort called with ([],), {} took 5.300000339047983e-06 seconds.

Selection Sort:
Function selection_sort called with ([],), {} took 4.800000169780105e-06 seconds.

----------------------------------------------------------------------------------------------------

Test Case 2: [1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2, 3, 4, 5, 1, 2,

In the above code, inside the `log_execution(func)` function, observe that the `wrapper(*args, **kwargs)` function is decorated with `@functool.wraps(func)`.

What is `@functools.wraps(func)`? Let us first try to understand it by writing another piece of code.

```Python
def outer(func):
    def wrapper(*args, **kwargs):
        print(func.__name__ + ' will be called after printing this particular line!')
        result = func(*args, **kwargs)
        print(func.__name__ + ' has been called and executed! The result will be printed on the next line.')
        return result
    return wrapper

@outer
def foo(x):
    '''
    Docstring of foo: This function foo computes the factorial of a number.
    '''
    fact = 1
    for i in range(1, x + 1):
        fact = fact * i
    return fact

print(foo(5))
```



The output of the `print` statement will be: <br>
```
foo will be called after printing this particular line!
foo has been called and executed! The result will be printed on the next line.
120
```

Let us now print the name and the docstring of the function directly and observe what we get.

```Python
print(foo.__name__) # Let us try to find out what output we will get when we try to print the name of foo using the __name__ dunder method.

print(foo.__doc__) # Let us try to find out what output we will get when we try to print the docstring using the __doc__ dunder method.
```


The output of the above `print` statements will be: <br>
```
wrapper
None
```

Let us now try to view the documentation of the function `foo(x)` by using the `help()` function.

```Python
help(foo)
```

The output will be:

```
Help on function wrapper in module __main__:

wrapper(*args, **kwargs)
```

Why are we not able to retrieve the name, docstring and the documentation of the function `foo(x)` even when we are using explicit dunder methods to do so?

The reason is, the function `foo(x)` has been replaced with the function `wrapper(*args, **kwargs)` because the function `foo(x)` has been decorated with `@outer`.

In Python, decorators are used to modify the behavior of a function or method. When a decorator wraps a function, it essentially replaces the original function with a new function (`foo(x)` is replaced by the `wrapper(*args, **kwargs)` function in this case) that adds additional functionality.

Hence we are not able retrieve the name, docstring and the documentation of `foo(x)`.

What if we want to preserve all the above mentioned details of the function `foo(x)`?

This is where `@functools.wraps(func)` comes in. 

`@functools.wraps(func)` is a decorator applied to the wrapper function. Its purpose is to update the wrapper function to look more like the original `func` function by copying attributes such as `__name__`, `__doc__`, etc., from `func` to `wrapper`. This helps in preserving metadata of the original function, which is useful for debugging and introspection.

By using `@functools.wraps(func)`, methods like `help()` and tools such as IDEs, can correctly identify the decorated function's signature and documentation.

Without `@functools.wraps(func)`, the wrapper function would not retain the name and docstring of the original func, which can be confusing and less informative when debugging or using documentation tools.

The output of the code given in this section with and without the use of `@functools.wraps(func)` has been demonstrated below:

### Without `@functools.wraps(func)`

In [14]:
def outer(func):
    def wrapper(*args, **kwargs):
        print(func.__name__ + ' will be called after printing this particular line!')
        result = func(*args, **kwargs)
        print(func.__name__ + ' has been called and executed! The result will be printed on the next line.')
        return result
    return wrapper

@outer
def foo(x):
    '''
    Docstring of foo: This function foo computes the factorial of a number.
    '''
    fact = 1
    for i in range(1, x + 1):
        fact = fact * i
    return fact
    
print(foo(5))
print('-' * 50)
print(foo.__name__)
print(foo.__doc__)
print('-' * 50)
help(foo)

foo will be called after printing this particular line!
foo has been called and executed! The result will be printed on the next line.
120
--------------------------------------------------
wrapper
None
--------------------------------------------------
Help on function wrapper in module __main__:

wrapper(*args, **kwargs)



### With `@functools.wraps(func)`

In [15]:
def outer(func):
    @functools.wraps(func)
    def wrapper(*args, **kwargs):
        print(func.__name__ + ' will be called after printing this particular line!')
        result = func(*args, **kwargs)
        print(func.__name__ + ' has been called and executed! The result will be printed on the next line.')
        return result
    return wrapper

@outer
def foo(x):
    '''
    Docstring of foo: This function foo computes the factorial of a number.
    '''
    fact = 1
    for i in range(1, x + 1):
        fact = fact * i
    return fact
    
print(foo(5))
print('-' * 50)
print(foo.__name__)
print(foo.__doc__)
print('-' * 50)
help(foo)

foo will be called after printing this particular line!
foo has been called and executed! The result will be printed on the next line.
120
--------------------------------------------------
foo

    Docstring of foo: This function foo computes the factorial of a number.
    
--------------------------------------------------
Help on function foo in module __main__:

foo(x)
    Docstring of foo: This function foo computes the factorial of a number.



## Problem 6: Access Control Decorator

<span style="font-size:20px;">Create a decorator that restricts access to a function based on a user role.</span>

In [16]:
class ItemNotFoundError(Exception):
    def __init__(self, item, message = 'Item not found in dictionary'):
        self.item = item
        self.message = message
        super().__init__(self.message)

    def __str__(self):
        return f'{self.message}: {self.item}'

# Declare a sample database to work with.
users_database = {
    'Alice': 'Admin',
    'Bob': 'Moderator',
    'Charlie': 'Guest',
    'Mark': 'Editor',
    'John': 'Contributor',
    'Jane': 'Subscriber',
    'Smith': 'Admin',
    'Ford': 'Moderator',
    'Miller': 'Guest'
}

# Solution to the problem starts here.
def get_user_role(username):
    return users_database.get(username)

def restrict_access(allowed_roles):
    def decorator(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            username = kwargs.get('username')
            given_user_role = kwargs.get('role')
            stored_user_role = get_user_role(username)

            if username not in users_database.keys():
                raise ItemNotFoundError(username)

            if (username in users_database.keys()) and (given_user_role != stored_user_role):
                raise ValueError(f'User \'{username}\' does not have the role of \'{given_user_role}\' in the database.')

            if stored_user_role not in allowed_roles:
                raise PermissionError(f'User \'{username}\' with role \'{stored_user_role}\' is not allowed to carry out this operation.')

            return func(*args, **kwargs)
        return wrapper
    return decorator

In [17]:
@restrict_access('Admin')
def add_user(username = None, role = None):
    new_username = input('Enter the name of the new user: ')
    new_role = input('Enter the role of the new user: ')
    users_database[new_username] = new_role
    print('New User Added' + '\n')
    return users_database

In [18]:
@restrict_access(['Admin', 'Moderator'])
def delete_user(username = None, role = None):
    to_del_user = input('Enter the name of the user to be deleted: ')

    if to_del_user not in users_database.keys():
        return f'{to_del_user} not found in the database.'

    if to_del_user == username:
        return f'You cannot delete yourself.'

    if role == 'Moderator' and users_database[to_del_user] == 'Admin':
        return f'A moderator cannot delete an admin.'

    print('User Deleted!' + '\n')

    users_database.pop(to_del_user)

    return users_database

In [19]:
add_user(username = 'Smith', role = 'Admin')

Enter the name of the new user:  Steve
Enter the role of the new user:  Moderator


New User Added



{'Alice': 'Admin',
 'Bob': 'Moderator',
 'Charlie': 'Guest',
 'Mark': 'Editor',
 'John': 'Contributor',
 'Jane': 'Subscriber',
 'Smith': 'Admin',
 'Ford': 'Moderator',
 'Miller': 'Guest',
 'Steve': 'Moderator'}

In [20]:
add_user(username = 'Alice', role = 'Admin')

Enter the name of the new user:  Amy
Enter the role of the new user:  Guest


New User Added



{'Alice': 'Admin',
 'Bob': 'Moderator',
 'Charlie': 'Guest',
 'Mark': 'Editor',
 'John': 'Contributor',
 'Jane': 'Subscriber',
 'Smith': 'Admin',
 'Ford': 'Moderator',
 'Miller': 'Guest',
 'Steve': 'Moderator',
 'Amy': 'Guest'}

In [21]:
delete_user(username = 'Ford', role = 'Moderator')

Enter the name of the user to be deleted:  Mark


User Deleted!



{'Alice': 'Admin',
 'Bob': 'Moderator',
 'Charlie': 'Guest',
 'John': 'Contributor',
 'Jane': 'Subscriber',
 'Smith': 'Admin',
 'Ford': 'Moderator',
 'Miller': 'Guest',
 'Steve': 'Moderator',
 'Amy': 'Guest'}